In [1]:
import pandas as pd

def get_training_data(df, smol=0.01, seed=1337):
    return df.sample(frac=smol, replace=False, random_state=1337)

h1b_data = pd.read_csv('data.csv')
desired_cols = ['JOB_TITLE',
                'SOC_NAME',
                'EMPLOYER_NAME',
                'WORKSITE_STATE',
                'WORKSITE_CITY',
                'WAGE_RATE_OF_PAY_FROM']
a = h1b_data[desired_cols]
a['WAGE_RATE_OF_PAY_FROM'] = a['WAGE_RATE_OF_PAY_FROM'].apply(lambda x: int(x.replace(',','')[:-3]))
c = a[~(a['WAGE_RATE_OF_PAY_FROM'] < 10000)].dropna().copy()
TECHIES = set(['SOFTWARE', 'PROGRAMMER', 'DEVELOPER', 'ENGINEER'])

from numpy.random import rand, seed
seed(1337)

# set prior to 1 as we use all TECHIE instances for feature creation
TECHIE_PRIOR = 1
is_a_techie = lambda job_title: any([word in TECHIES for word in job_title.upper().strip().split()]) and rand() <= TECHIE_PRIOR
c['TECHIE'] = c['JOB_TITLE'].map(is_a_techie)

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [2]:
job_freqs = c[['JOB_TITLE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count()

# I named this variable after knowing 93 would be the number of cities with > 1000 jobs
top_93 = job_freqs[job_freqs['JOB_TITLE']>1000].sort_values('JOB_TITLE', ascending=False)
print(len(top_93))
top_93.head()

93


,JOB_TITLE
WORKSITE_CITY,
NEW YORK,29700
SAN FRANCISCO,13050
HOUSTON,11039
CHICAGO,9534
ATLANTA,9120


In [3]:
top_93['WORKSITE_CITY'] = top_93.index
print(len(c))
print(len(c[c['TECHIE']==True]))
print(len(c[c['TECHIE']==False]))
total_techies = len(c[c['TECHIE']==True])

494762
219428
275334


In [4]:
top_93_cities_series = top_93['WORKSITE_CITY']
top_93_cities = set(top_93_cities_series.tolist())

In [5]:
d = c[c['WORKSITE_CITY'].isin(top_93_cities)]

In [38]:
city_weights = d[d['TECHIE']==True][['TECHIE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count() / total_techies
city_weights['WEIGHTS'] = city_weights.apply(lambda x: x/x.max(), axis=0)
city_weights['WORKSITE_CITY'] = city_weights.index

# # to0 lazy
# GOOGLE_MAPS_API_KEY = open('.env').read().split('=')[1].strip()

# import requests
# import json
# get_lat_lng = lambda city: json.loads(requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}'.format(city.replace(' ', '%20'), GOOGLE_MAPS_API_KEY)).text)['results'][0]['geometry']['location']
# get_lat = lambda city: get_lat_lng(city)['lat']
# get_lng = lambda city: get_lat_lng(city)['lng']
# city_weights['LAT'] = city_weights['WORKSITE_CITY'].map(get_lat)
# city_weights['LNG'] = city_weights['WORKSITE_CITY'].map(get_lng)
# city_weights.head()

In [20]:

city_lat_lng_df = pd.read_csv('city_lat_lng.csv', sep='\t')
city_lat_lng_df['LAT']
city_weights['WORKSITE_CITY']
city_lat_lng_df.index = city_lat_lng_df['CITY']
city_weights['LAT'] = city_lat_lng_df['LAT']
city_weights['LNG'] = city_lat_lng_df['LNG']

city_weights.head()


# weight_map = city_weights.to_dict()['TECHIE']
# d['WEIGHTS'] = d['WORKSITE_CITY'].map(weight_map)
# d.head()

,TECHIE,WEIGHTS,WORKSITE_CITY,LAT,LNG
WORKSITE_CITY,,,,,
ALPHARETTA,0.010783,0.380937,ALPHARETTA,34.075376,-84.294090
ATLANTA,0.015107,0.533730,ATLANTA,33.748995,-84.387982
AUBURN HILLS,0.004001,0.141362,AUBURN HILLS,42.687532,-83.234103
AUSTIN,0.014146,0.499758,AUSTIN,30.267153,-97.743061
BALTIMORE,0.001782,0.062953,BALTIMORE,39.290385,-76.612189


In [116]:
city_lat_lng_df[30:60]

,CITY,LAT,LNG
CITY,,,
FRISCO,FRISCO,33.150674,-96.823612
HARTFORD,HARTFORD,41.763711,-72.685093
HERNDON,HERNDON,38.969555,-77.386098
HILLSBORO,HILLSBORO,45.522894,-122.989827
HOUSTON,HOUSTON,29.760427,-95.369803
INDIANAPOLIS,INDIANAPOLIS,39.768403,-86.158068
IRVINE,IRVINE,33.684567,-117.826505
IRVING,IRVING,32.814018,-96.948894
ISELIN,ISELIN,40.575382,-74.322370


In [33]:

lats = pd.Series(city_weights.LAT.values, index=city_weights.WORKSITE_CITY).to_dict()
lngs = pd.Series(city_weights.LNG.values, index=city_weights.WORKSITE_CITY).to_dict()

# {city_weights['WORKSITE_CITY'] : city_weights['LAT'] for city in city_weights.keys()}
# del city_weights['TECHIE']
# del city_weights['WORKSITE_CITY']
# city_weights.to_csv('lat_lng_weights.tsv', sep='\t', index=False, header=False)


In [43]:

city_weights
cities_to_weights = pd.Series(city_weights.WEIGHTS.values, index=city_weights.WORKSITE_CITY).to_dict()
cities_to_weights
d.head()

,JOB_TITLE,SOC_NAME,EMPLOYER_NAME,WORKSITE_STATE,WORKSITE_CITY,WAGE_RATE_OF_PAY_FROM,TECHIE
2,.NET SOFTWARE PROGRAMMER,COMPUTER PROGRAMMERS,EASTBANC TECHNOLOGIES LLC,DC,WASHINGTON,77000,True
3,PROJECT MANAGER,"COMPUTER OCCUPATIONS, ALL OTHER",INFO SERVICES LLC,NJ,JERSEY CITY,102000,False
4,ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES,CREDIT ANALYSTS,BB&T CORPORATION,NY,NEW YORK,132500,False
5,CREDIT RISK METRICS SPECIALIST,"FINANCIAL SPECIALISTS, ALL OTHER","SUNTRUST BANKS, INC.",GA,ATLANTA,71750,False
6,BUSINESS SYSTEMS ANALYST,MANAGEMENT ANALYSTS,CITADEL INFORMATION SERVICES INC.,NJ,EDISON,61000,False


In [44]:
d['WEIGHTS'] = d['WORKSITE_CITY'].map(cities_to_weights)
desired_cols = ['LAT', 'LNG', 'WEIGHTS']
# city
e = d.copy()
e['LAT'] = d['WORKSITE_CITY'].map(lats)
e['LNG'] = d['WORKSITE_CITY'].map(lngs)
f = e[desired_cols]
f.head()

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,LAT,LNG,WEIGHTS
2,47.751074,-120.740139,0.147641
3,40.728158,-74.077642,0.247947
4,40.712784,-74.005941,1.000000
5,33.748995,-84.387982,0.533730
6,40.518715,-74.412095,0.234745


In [53]:
f.to_csv('kmeans_vectors.tsv', sep='\t', header=False, index=False)
f = f.dropna()

In [54]:
vectors = f.as_matrix()

In [55]:
from kmeans import kmeans, get_henris

In [56]:
centroids, C = kmeans(vectors, 5)

/Users/rainierababao/cs/hackmit/deeplearning/kmeans.py:12: RuntimeWarning: Mean of empty slice.
  centroids = [features[cluster == k].mean(axis = 0) for k in range(k)]
/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [57]:
centroids

array([[  34.73907742,  -89.14790687,    0.35742674],
       [  41.40067058,  -72.19843494,    0.46364779],
       [  36.64149302, -120.02558448,    0.50384592],
       [  47.41029451, -122.1012449 ,    0.38591232],
       [          nan,           nan,           nan]])

In [58]:
d.head()

,JOB_TITLE,SOC_NAME,EMPLOYER_NAME,WORKSITE_STATE,WORKSITE_CITY,WAGE_RATE_OF_PAY_FROM,TECHIE,WEIGHTS
2,.NET SOFTWARE PROGRAMMER,COMPUTER PROGRAMMERS,EASTBANC TECHNOLOGIES LLC,DC,WASHINGTON,77000,True,0.147641
3,PROJECT MANAGER,"COMPUTER OCCUPATIONS, ALL OTHER",INFO SERVICES LLC,NJ,JERSEY CITY,102000,False,0.247947
4,ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES,CREDIT ANALYSTS,BB&T CORPORATION,NY,NEW YORK,132500,False,1.000000
5,CREDIT RISK METRICS SPECIALIST,"FINANCIAL SPECIALISTS, ALL OTHER","SUNTRUST BANKS, INC.",GA,ATLANTA,71750,False,0.533730
6,BUSINESS SYSTEMS ANALYST,MANAGEMENT ANALYSTS,CITADEL INFORMATION SERVICES INC.,NJ,EDISON,61000,False,0.234745


In [70]:
d_techies = d[d['TECHIE']]
d_techies.head()

,JOB_TITLE,SOC_NAME,EMPLOYER_NAME,WORKSITE_STATE,WORKSITE_CITY,WAGE_RATE_OF_PAY_FROM,TECHIE,WEIGHTS
2,.NET SOFTWARE PROGRAMMER,COMPUTER PROGRAMMERS,EASTBANC TECHNOLOGIES LLC,DC,WASHINGTON,77000,True,0.147641
7,PROGRAMMER ANALYST,COMPUTER PROGRAMMERS,CITADEL INFORMATION SERVICES INC.,NJ,EDISON,60500,True,0.234745
8,PROGRAMMER ANALYST,COMPUTER PROGRAMMERS,CITADEL INFORMATION SERVICES INC.,NY,NEW YORK,60400,True,1.000000
9,PROGRAMMER ANALYST,COMPUTER PROGRAMMERS,CITADEL INFORMATION SERVICES INC.,NY,NEW YORK,60450,True,1.000000
20,PROGRAMMER ANALYST,COMPUTER SYSTEMS ANALYSTS,"IDC TECHNOLOGIES, INC.",CA,MILPITAS,104000,True,0.165513


In [71]:
ee = d_techies.copy()
ee['LAT'] = d_techies['WORKSITE_CITY'].map(lats)
ee['LNG'] = d_techies['WORKSITE_CITY'].map(lngs)
ff = ee[desired_cols]
ff.head()

,LAT,LNG,WEIGHTS
2,47.751074,-120.740139,0.147641
7,40.518715,-74.412095,0.234745
8,40.712784,-74.005941,1.000000
9,40.712784,-74.005941,1.000000
20,37.432334,-121.899574,0.165513


In [72]:
ff.to_csv('kmeans_vectors_techies_only.tsv', sep='\t', header=False, index=False)

In [76]:
vectors_techies = ff.as_matrix()
centroids, C = kmeans(vectors_techies, 5)

In [77]:
centroids

array([[  37.48111736, -122.09048306,    0.59388768],
       [  47.40286435, -122.16740711,    0.39779135],
       [  38.92911133,  -78.21396681,    0.30300106],
       [  33.7999066 , -115.38290067,    0.24968074],
       [  32.51870328,  -96.60645509,    0.44500609]])

In [78]:
ff.to_csv('kmeans_vectors_techies_only.tsv', sep='\t', header=False, index=False)

In [81]:
df0 = pd.DataFrame({'LAT': centroids[0][0],
       'LNG' : centroids[0][1],
       'WEIGHTS' : centroids[0][2]}, index=[0])
frames = [df0, f]
result = pd.concat(frames)

In [82]:
result.head()

,LAT,LNG,WEIGHTS
0,37.481117,-122.090483,0.593888
2,47.751074,-120.740139,0.147641
3,40.728158,-74.077642,0.247947
4,40.712784,-74.005941,1.000000
5,33.748995,-84.387982,0.533730


In [86]:
points, mappings = kmeans(result.as_matrix(), 5)

In [87]:
points

array([[  40.46173925,  -74.5300538 ,    0.16491004],
       [  42.02506854,  -67.75997803,    0.7489692 ],
       [  33.89161207,  -79.42847264,    0.17024544],
       [  38.10448747,  -83.78389443,    0.27049114],
       [  37.41114272, -110.12279919,    0.45833773]])

In [90]:
henris = get_henris(mappings)

In [91]:
henris

[0,
 1,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 36,
 37,
 38,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 58,
 60,
 61,
 62,
 63,
 64,
 66,
 67,
 68,
 83,
 86,
 87,
 89,
 91,
 92,
 95,
 96,
 99,
 106,
 108,
 111,
 112,
 115,
 117,
 118,
 121,
 122,
 127,
 128,
 130,
 131,
 132,
 133,
 135,
 137,
 138,
 139,
 140,
 142,
 144,
 146,
 150,
 152,
 153,
 154,
 156,
 157,
 159,
 160,
 162,
 164,
 166,
 171,
 172,
 173,
 174,
 176,
 177,
 180,
 181,
 182,
 184,
 185,
 186,
 190,
 192,
 193,
 195,
 196,
 197,
 198,
 199,
 200,
 202,
 203,
 206,
 208,
 209,
 212,
 213,
 215,
 216,
 217,
 219,
 220,
 222,
 223,
 225,
 227,
 230,
 231,
 233,
 235,
 237,
 240,
 242,
 244,
 245,
 247,
 249,
 250,
 252,
 254,
 257,
 258,
 262,
 263,
 265,
 266,
 267,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 284,
 286,
 287,
 288,
 289,
 291,
 292,
 293,
 294,
 295,
 306,
 307,
 309,
 311,
 315,
 317,
 318,
 319,
 320,
 321,
 322,
 3

In [93]:
henri_pts = [result.iloc[x] for x in henris]

In [101]:
henri_pts[0]

LAT         37.481117
LNG       -122.090483
WEIGHTS      0.593888
Name: 0, dtype: float64

In [105]:
write_this = list(map(lambda x: [x['LAT'], x['LNG'], x['WEIGHTS']], henri_pts))

In [108]:
with open("henri_ca.csv", "w") as f:
    for line in write_this:
        f.write('{},{},{}\n'.format(line[0], line[1], line[2]))

In [ ]:
df0 = pd.DataFrame({'LAT': centroids[0][0],
       'LNG' : centroids[0][1],
       'WEIGHTS' : centroids[0][2]}, index=[0])
frames = [df0, f]
result = pd.concat(frames)